# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

**Задание 1. Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('data2.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` заполним медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Задание 8. Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**Задание 9. В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Задание 11. Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Испольуем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)


### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:



data_pivot = data.pivot_table(index = ['children'], values = 'debt').round(3)
data_pivot=data_pivot.sort_values(by='debt',ascending=True)
data_pivot.loc[:,'debt']=data_pivot['debt'].map('{:.2%}'.format)


data_pivot.head(6)


,debt
children,
5,0.00%
0,7.50%
3,8.20%
1,9.20%
2,9.50%
4,9.80%


**3.1.1 Посмотрим размеры групп столбца "children"**

In [23]:
data.groupby('children')['children'].count()

children
0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

 # Вывод.

**Бездетные, исходя из данных таблицы, более ответственно подходят к своевременному погашению кредита. Клиенты с 1 и 2-мя и 4-мя детьми реже платят в срок.Данные по клиентам имеющим 5 детей не однозначны, так как их выборка очень мала,их всего 9 человек. Клиенты с 3-мя детьми чаще платят в срок чем другие, имеющие детей клиенты.**

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
               

data_pivot = data.pivot_table(index = ['family_status'], values = 'debt').round(3)
data_pivot=data_pivot.sort_values(by='debt',ascending=True)
data_pivot.loc[:,'debt']=data_pivot['debt'].map('{:.2%}'.format)
data_pivot.head(6)                          


,debt
family_status,
вдовец / вдова,6.60%
в разводе,7.10%
женат / замужем,7.60%
гражданский брак,9.30%
Не женат / не замужем,9.80%


# Вывод.

**Если клиент  состоял когда либо в браке и на данный момент разведен или вдовствует,то вероятнось возврата кредита в срок выше на 0.5-3.2 п.п.**



#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
data_pivot = data.pivot_table(index = ['total_income_category'], values = 'debt').round(3)
data_pivot=data_pivot.sort_values(by='debt',ascending=True)
data_pivot.loc[:,'debt']=data_pivot['debt'].map('{:.2%}'.format)
data_pivot.head(6)         # Ваш код будет здесь. Вы можете создавать новые ячейки.

,debt
total_income_category,
D,6.00%
B,7.10%
A,8.00%
C,8.50%
E,9.10%


**3.3.1 Определим количество клиентов в каждой группе столбца "total_income_category".**

In [26]:

data.groupby('total_income_category')['total_income_category'].count()

total_income_category
A       25
B     5014
C    15921
D      349
E       22
Name: total_income_category, dtype: int64

# Вывод.
**Группа "Е" - клиенты с доходом до 30000 рублей (категория Е)  самая малочисленная (22 человека), так же как и группа с "А"- клиенты с доходом 1000001 и выше(25 человек). Скорее всего это связано с тем, что клиентов группы "Е" банк предпочитает не кредитовать,так-как их дохода едва хватает на покрытие базовых потребностей. А группа клиентов "А"сама по себе не может быть многочисленной из-за экономичкских условий. Поэтому эти две группы не будем учитывать в анализе. Самыми же ответственными плательщиками из оставшихся трех групп являются клиенты с доходом 30001-50000 рублей(категория D), с 6.00% невозврата по кредитам.Клиенты с более высокими доходами категории "В"(200001-1000000) рублей имеют 7.10 % невозврата и самые ненадежные заемщики из рассматриваемых групп - категория "С"50001-200000)рублей с 8.50% невозврата.**


#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [27]:
data_pivot = data.pivot_table(index = ['purpose_category'], values = 'debt').round(3)
data_pivot=data_pivot.sort_values(by='debt',ascending=True)
data_pivot.loc[:,'debt']=data_pivot['debt'].map('{:.2%}'.format)
data_pivot.head(6)  # Ваш код будет здесь. Вы можете создавать новые ячейки.

,debt
purpose_category,
операции с недвижимостью,7.30%
проведение свадьбы,7.90%
операции с автомобилем,9.30%
получение образования,9.30%


# Вывод.
**Кредиты, взятые на операции с недвижимостью и проведение свадьбы имеют более низкий процент невозврата в срок(7.30% и 7.90%) чем кредиты взятые на операции с автомобилем и получение образования(9.30% и 9.30%)что соответстввует разнице на 2 п.п. и 1.4 п.п.**

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.


**В двух столбцах есть пропущенные значения. Один из них — days_employed - общий трудовой стаж. Другой столбец с пропущенными значениями — total_income — хранит данные о доходах. Возможно пропуски в данных имеют технический характер, так как имеют одинаковый показаель-2174. На сумму дохода сильнее всего влияет тип занятости, поэтому мы заполнили пропуски медианным значением по каждому типу из столбца income_type(тип занятости).**

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

**Пропуски в количественных переменных лучше всего заполнять медианным значением, так как медиана определяет центр распределения, т.е. область, в которой его значения наиболее вероятны.Наборы данных могут имеют выбросы в последовательности, которые могут привести к асимметрии среднего значения. Медиана последовательности на самом деле может быть меньше затронута этими выбросами по сравнению со средним.**



### Шаг 4: общий вывод.

Напишем общий вывод.

 # Общий вывод.
 
 
 **Идеальным клиентом считается: 
 Заемщик (мужчина или женщины) взявший кредит на операцию с недвижимостью(7.3% невозврата в срок),а самые безответственные заемщики - берущие кредит на операции с автомобилем и образование(9.30% невозврата), не имеющий детей клиенты имеют 7.50% невозврата кредитов,тогда как у  имеющих детей заемщиков этот процент выше и варьируется от 8.20%-9.80% ,клиенты с доходом от 30001-50000 рублей более ответственны, их процент невозврата-6.0%,самые ненадежные- категория 'С' с доходом 50001-200000 рублей(8.50%). Находящийся в разводе или вдовствующий заемщик,согласно исследованию,имеет самый низкий процент невозврата по кредиту-6.60%, тогда как клиенты которые никогда не состояли в браке- 9.80%. На мой взгляд, это связано с тем , что одинокому человеку проще контролировать свой бюджет.Отсутствие супруга и детей значительно уменьшают вероятность всякого рода непредвиденных расходов, что в свою очередь приводит к постоянству выплат по кредиту, в то же время семья в прошлом сделала его более ответственным, а те, кто никогда не состоял в браке, видимо, менее ответственны в планировании своего бюджета потому их процент невыплат на 3.20 п.п. выше ,чем у тех ,кто когда либо был в браке.**
